In [1]:
import os
import re
import requests
from bs4 import BeautifulSoup
import mimetypes
import pandas as pd
from pandas import DataFrame as df

In [3]:
# (파일의 경로)를 입력 받아 
# 해당 파일의 (파일 유형)을 리턴 (ex: 'photo')
def get_type_of_file_from_full_path(full_path):
    # 해당 경로가 파일 경로가 아닌 경우 None 리턴
    if not os.path.isfile(full_path):
        return None
    
    mime_type = mimetypes.guess_type(full_path)[0] # ('maintype/subtype',encoding)
    # 파일 유형을 찾지 못한 경우 None 리턴
    if mime_type == None:
        return None

    # 정규표현식을 써서 파일의 유형을 확인한다.
    patrn = re.compile('^(?P<maintype>.+)\/(?P<subtype>.+)$')
    m = patrn.search(mime_type)
    # 패턴 매칭이 안된 경우 None 리턴
    if m.group('maintype') == None:
        return None
    # 파일의 유형은 maintype이 말해준다.
    file_type = m.group('maintype')
    return file_type
    
full_path = r'C:\Users\darau\Desktop\Python Projects\WebCrawl\[Geforce] RTX-3090 RTX 3090B.mp4'
get_type_of_file_from_full_path(full_path)

'video'

In [4]:
# (폴더 경로)를 입력 받아
# 해당 폴더 아래의 모든 비디오 파일을 검색.
# 검색된 모든 비디오 파일들의 경로를 리턴
def get_video_path_list_from_dir_path(dir_path,video_path_list=[]):
    if not os.path.isdir(dir_path):
        return None
    # 폴더 내부의 모든 파일 및 경로를 따라가며
    for path in os.listdir(dir_path):
        # 경로가 파일 경로인 경우
        if os.path.isfile(path):
            # 파일이 비디오 유형인 경우
            if get_type_of_file_from_full_path(path) == 'video':
                video_path_list.append(path)
        # 경로가 폴더 경로인 경우
        elif os.path.isdir(path):
            get_video_path_list_from_dir_path(path,video_path_list)
    # 폴더 내부의 
    return video_path_list            

dir_path = r'C:\Users\darau\Desktop\Python Projects\WebCrawl'
get_video_path_list_from_dir_path(dir_path)

['RTX-3090B.mp4', '[Geforce] RTX-3090 RTX 3090B.mp4']

In [25]:
# 파일의 이름을 입력받아
# 정규화된 파일 이름을 리턴.
def regularize_file_name(file_name):
    file_base, file_ext = os.path.splitext(file_name) # (file_base, .file_ext)
    patrn = re.compile('.*?(?P<prefix>[a-zA-Z]{3,6})[-](?P<number>\d{3,4}).*?(?P<suffix>[A-J])?$')
    m = patrn.search(file_base)
    # 정규표현식 매칭이 성공할 경우
    if m != None:
        # 새 파일 이름을 정의
        prefix = m.group('prefix')
        number = m.group('number')
        # 접미사가 없는 경우
        if m.group('suffix') == None:
            new_file_base = prefix+'-'+number
            new_file_name = new_file_base+file_ext
        # 접미사가 있는 경우
        else:
            suffix = m.group('suffix')
            new_file_base = prefix+'-'+number+suffix
            new_file_name = new_file_base+file_ext
        return new_file_name

regularize_file_name('[Geforce] RTX-3090 RTX 3090B.mp4')


'RTX-3090B.mp4'

In [5]:
# 폴더 경로를 입력 받아
# 해당 폴더 아래의 모든 파일들의 경로를 리턴
def get_file_path_list_from_dir_path(dir_path):
    file_path_list=[]
    for (root, directories, files) in os.walk(dir_path):
        for d in directories:
            d_path = os.path.join(root, d)
            file_path_list.append(d_path)

        for file in files:
            file_path = os.path.join(root, file)
            file_path_list.append(file_path)
    # # 폴더 내부의 모든 파일 및 경로를 따라가며
    # for path in os.listdir(dir_path):
    #     # 경로가 파일 경로인 경우 파일 경로 리스트에 추가
    #     if os.path.isfile(path):
    #         full_path = os.path.join(dir_path,path)
    #         file_path_list.append(full_path)
    #     # 경로가 폴더 경로인 경우
    #     elif os.path.isdir(path):
    #         get_file_path_list_from_dir_path(path,file_path_list)
    # # 폴더 내부의 
    # return file_path_list  

dir_path = r'C:\Users\darau\Dropbox\Python\WebCrawl'
get_file_path_list_from_dir_path(dir_path)

In [ ]:
# 폴더 경로를 입력 받아
# 폴더 내의 모든 파일들을 정규화된 이름으로 수정.
def edit_file_name_in_dir(dir_path):
    file_path_list = [] 
    file_path_list = get_file_path_list_from_dir_path(dir_path)
    return file_path_list
    # for full_path in file_path_list:
    #     file_dir, file_name = os.path.split(full_path)
    #     new_file_name = regularize_file_name(file_name)
    #     if new_file_name != None:
    #         new_full_path = os.path.join(file_dir,new_file_name)
            # print(new_full_path)

dir_path = r'C:\Users\darau\Dropbox\Python\WebCrawl'
edit_file_name_in_dir(dir_path)


In [30]:
def download_from_url(url, file_name):
    with open(file_name, "wb") as file:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        response = requests.get(url, headers = headers)
        file.write(response.content) 

In [37]:
def download_from_prod_id(prod_id):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    url = f'https://javdb.com/search?q={prod_id}&f=all'
    req = requests.get(url, headers = headers)
    soup = BeautifulSoup(req.content,"html.parser")
    reg_name_source = soup.find("img",attrs = {"lazy"})

    patrn = re.compile('thumbs/(?P<keyword>.{3,}).jpg" src=')
    m = patrn.search(str(reg_name_source))
    keyword = m.group("keyword")
    url = f'https://jdbimgs.com/covers/{keyword}.jpg'
    req = requests.get(url, headers = headers)
    dir_path = r'.\crawled_img'
    full_path = dir_path+'\\'+f'{prod_id}.jpg'

    f = open(full_path, 'wb')
    f.write(req.content)
    f.close()


In [138]:
download_from_prod_id('gachi-1151')

In [1]:
[1,2,3]==[3,2,1]

False

In [2]:
set([1,2,3])==set([3,2,1])

True

In [9]:
PID = 'STAR-550'
url = f'https://jav.land/en/id_search.php?keys={PID}'
req = requests.get(url)
soup = BeautifulSoup(req.content,"html.parser")
CID_source = soup.find("td", attrs = {"width":"80%"})
patrn = re.compile('^<td width="80%">(?P<CID>.{3,})<\/td>$')
m = patrn.search(str(CID_source))
if m!= None:
    print(m.group("CID"))
else:
    print(None)

1star00550


In [19]:
data = {'name': ['Beomwoo', 'Beomwoo', 'Beomwoo', 'Kim', 'Park'],
        'year': [2013, 2014, 2015, 2016, 2015],
        'points': [1.5, 1.7, 3.6, 2.4, 2.9]}
metadata = pd.DataFrame(data)

In [20]:
print(metadata)

      name  year  points
0  Beomwoo  2013     1.5
1  Beomwoo  2014     1.7
2  Beomwoo  2015     3.6
3      Kim  2016     2.4
4     Park  2015     2.9


In [44]:
data_csv = pd.read_csv(r'./metadata.csv')
num_row = data_csv.shape[0]
print(num_row)
for index in range(num_row):
    print(tuple(data_csv.loc[index]))


1113
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/JAV/ABP-051.wmv')
('ABP-051', nan, nan, 'X:/J

nan

In [1]:
import tkinter
import tkinter.ttk

window=tkinter.Tk()
window.title("YUN DAE HEE")
window.geometry("640x400+100+100")
window.resizable(False, False)

def cc(self):
    treeview.tag_configure("tag2", background="red")

treeview=tkinter.ttk.Treeview(window, columns=["one", "two"], displaycolumns=["two", "one"])
treeview.pack()

treeview.column("#0", width=70)
treeview.heading("#0", text="num")

treeview.column("one", width=100, anchor="center")
treeview.heading("one", text="문자", anchor="e")

treeview.column("#2", width=100, anchor="w")
treeview.heading("two", text="ASCII", anchor="center")

treelist=[("A", 65), ("B", 66), ("C", 67), ("D", 68), ("E", 69)]

for i in range(len(treelist)):
    
    treeview.insert('', 'end', text=i, values=treelist[i], iid=str(i)+"번")

top=treeview.insert('', 'end', text=str(len(treelist)), iid="5번", tags="tag1")
top_mid1=treeview.insert(top, 'end', text="5-2", values=["SOH", 1], iid="5번-1")
top_mid2=treeview.insert(top, 0, text="5-1", values=["NUL", 0], iid="5번-0", tags="tag2")
top_mid3=treeview.insert(top, 'end', text="5-3", values=["STX", 2], iid="5번-2", tags="tag2")

treeview.tag_bind("tag1", sequence="<<TreeviewSelect>>", callback=cc)

window.mainloop()

In [24]:
mylist = [1,2,3]
mylist[2:0:-1]

[3, 2]

In [3]:
int(3.4)

3

In [21]:
import math
math.ceil(34/5)

7

In [15]:
row_size, column_size = 3,4
[[ (row,column) for column in range(column_size)] for row in range(row_size)]

[[(0, 0), (0, 1), (0, 2), (0, 3)],
 [(1, 0), (1, 1), (1, 2), (1, 3)],
 [(2, 0), (2, 1), (2, 2), (2, 3)]]